In [1]:
# Instale as bibliotecas
%pip install pandas sentence-transformers faiss-cpu transformers spacy

In [2]:
import pandas as pd

# Carregar o arquivo CSV
df = pd.read_csv('chat_data.csv')

# Exibir os dados carregados
df.head()

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958.0,F,RJ
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996.0,M,SC
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984.0,M,SP
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985.0,F,SP
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994.0,M,MG


In [3]:
!python -m spacy download pt_core_news_sm

     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     ------------ --------------------------- 4.2/13.0 MB 22.9 MB/s eta 0:00:01
     -------------------------- ------------- 8.7/13.0 MB 21.5 MB/s eta 0:00:01
     --------------------------------------  12.8/13.0 MB 21.2 MB/s eta 0:00:01
     --------------------------------------- 13.0/13.0 MB 19.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [4]:
import pandas as pd
import spacy

# Carregar o modelo do spaCy para pt
nlp = spacy.load('pt_core_news_sm')

# Função de pré-processamento
def preprocess_text(text):
    if pd.isna(text):  # Verificar se o texto não é número
        return ''
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Carregar o CSV
df = pd.read_csv('chat_data.csv')

# Identificar colunas de texto dentro da base de dados para processamento
text_columns = ['product_name', 'product_brand', 'site_category_lv1', 'site_category_lv2', 'review_title', 'recommend_to_a_friend', 'review_text', 'reviewer_gender', 'reviewer_state']  # Liste aqui as colunas de texto

# Aplicar o pré-processamento a todas as colunas de texto
for col in text_columns:
    df[f'{col}_processado'] = df[col].apply(preprocess_text)

# Visualizar os dados pré-processados
print(df.head())

# Salvar o DataFrame processado em um novo arquivo CSV
df.to_csv('chat_data_processado.csv', index=False)


       submission_date                                        reviewer_id  \
0  2018-01-01 00:11:28  d0fb1ca69422530334178f5c8624aa7a99da47907c44de...   
1  2018-01-01 00:13:48  014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...   
2  2018-01-01 00:26:02  44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...   
3  2018-01-01 00:35:54  ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...   
4  2018-01-01 01:00:28  7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...   

   product_id                                       product_name  \
0   132532965  Notebook Asus Vivobook Max X541NA-GO472T Intel...   
1    22562178               Copo Acrílico Com Canudo 500ml Rocie   
2   113022329  Panela de Pressão Elétrica Philips Walita Dail...   
3   113851581               Betoneira Columbus - Roma Brinquedos   
4   131788803  Smart TV LED 43" LG 43UJ6525 Ultra HD 4K com C...   

    product_brand      site_category_lv1       site_category_lv2  \
0             NaN            Informática                Note

In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Carregar o modelo Sentence-BERT
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Lista de colunas processadas
text_columns = ['product_name_processado', 'product_brand_processado', 'site_category_lv1_processado',
                'site_category_lv2_processado', 'review_title_processado', 'recommend_to_a_friend_processado',
                'review_text_processado', 'reviewer_gender_processado', 'reviewer_state_processado']

# Dicionário para armazenar embeddings de cada coluna
embeddings_dict = {}

# Iterar sobre as colunas de texto processado
for col in text_columns:
    if col in df.columns:
        # Preencher valores ausentes com strings vazias
        df[col] = df[col].fillna('')

        # Gerar embeddings para cada coluna de texto processado
        embeddings = model.encode(df[col].tolist())

        # Armazenar os embeddings no dicionário
        embeddings_dict[col] = embeddings

        # Verificar o formato dos embeddings para cada coluna
        print(f"Embeddings shape for {col}: {np.array(embeddings).shape}")
    else:
        print(f"A coluna '{col}' não existe no DataFrame.")

h:\Github\Fatec\BACK-DOMROCK\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
h:\Github\Fatec\BACK-DOMROCK\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Embeddings shape for product_name_processado: (124, 384)
Embeddings shape for product_brand_processado: (124, 384)
Embeddings shape for site_category_lv1_processado: (124, 384)
Embeddings shape for site_category_lv2_processado: (124, 384)
Embeddings shape for review_title_processado: (124, 384)
Embeddings shape for recommend_to_a_friend_processado: (124, 384)
Embeddings shape for review_text_processado: (124, 384)
Embeddings shape for reviewer_gender_processado: (124, 384)
Embeddings shape for reviewer_state_processado: (124, 384)


In [6]:
import faiss
import numpy as np

# Criar índice FAISS
dimension = embeddings.shape[1]  # Dimensão dos embeddings
index = faiss.IndexFlatL2(dimension)  # Distância L2

# Adicionar embeddings ao índice FAISS
index.add(np.array(embeddings))

# Verificar o número de vetores no índice
print(f"Número de vetores no índice: {index.ntotal}")

Número de vetores no índice: 124


In [7]:
import numpy as np
import faiss

# Função de busca usando FAISS
# Vai precisar aprimorar isso para garantir que a relação entre comentários está existindo
def buscar_no_faiss(consulta, tipo='comentario', k=1):
    """
    Realiza a busca no FAISS com base na consulta.
    
    :param consulta: A consulta de busca (texto ou número)
    :param tipo: O tipo de consulta ('comentario', 'produto', 'nota')
    :param k: Número de resultados desejados
    :return: Lista com os resultados encontrados
    """
    if tipo == 'nota':
        # Consultas por nota
        try:
            nota = float(consulta)
            resultados = df[df['overall_rating'] == nota].head(k)
            return resultados[['product_name', 'review_text', 'overall_rating']].to_dict(orient='records')
        except ValueError:
            print("Consulta de nota inválida.")
            return []

    elif tipo in ['comentario', 'produto']:
        # Pré-processar a consulta
        consulta_processada = preprocess_text(consulta)
        
        # Gerar embedding da consulta
        consulta_embedding = model.encode([consulta_processada])
        
        # Realizar a busca no índice FAISS
        distances, indices = index.search(np.array(consulta_embedding), k=k)
        
        # Exibir os 'k' resultados mais similares
        resultados = []
        for i in range(k):
            resultado = df.iloc[indices[0][i]]
            resultados.append({
                'produto': resultado['product_name'],
                'comentário': resultado['review_text'],
                'nota': resultado['overall_rating']
            })
        return resultados
    
    else:
        print("Tipo de consulta inválido.")
        return []

# Exemplos de consultas
consulta_nota = "4"  # Consulta por nota
resultados_nota = buscar_no_faiss(consulta_nota, tipo='nota', k=3)

consulta_comentario = "Produto razoável, pode melhorar."  # Consulta por comentário
resultados_comentario = buscar_no_faiss(consulta_comentario, tipo='comentario', k=3)

consulta_produto = "Notebook Asus Vivobook Max X541NA-GO472T Intel Celeron Quad Core 4GB 500GB Tela LED 15,6"  # Consulta por produto
resultados_produto = buscar_no_faiss(consulta_produto, tipo='produto', k=3)

# Exibir resultados
print("Resultados para consulta por nota:")
for i, res in enumerate(resultados_nota):
    print(f"Resultado {i+1}:")
    print(f"Produto: {res['product_name']}")
    print(f"Comentário: {res['review_text']}")
    print(f"Avaliação: {res['overall_rating']}\n")

print("Resultados para consulta por comentário:")
for i, res in enumerate(resultados_comentario):
    print(f"Resultado {i+1}:")
    print(f"Produto: {res['produto']}")
    print(f"Comentário: {res['comentário']}")
    print(f"Avaliação: {res['nota']}\n")

print("Resultados para consulta por produto:")
for i, res in enumerate(resultados_produto):
    print(f"Resultado {i+1}:")
    print(f"Produto: {res['produto']}")
    print(f"Comentário: {res['comentário']}")
    print(f"Avaliação: {res['nota']}\n")


Resultados para consulta por nota:
Resultado 1:
Produto: Notebook Asus Vivobook Max X541NA-GO472T Intel Celeron Quad Core 4GB 500GB Tela LED 15,6" Windows - 10 Branco
Comentário: Estou contente com a compra entrega rápida o único problema com as Americanas é se houver troca ou devolução do produto o consumidor tem problemas com espera.
Avaliação: 4

Resultado 2:
Produto: Copo Acrílico Com Canudo 500ml Rocie
Comentário: Por apenas R$1994.20,eu consegui comprar esse lindo copo de acrílico.
Avaliação: 4

Resultado 3:
Produto: Panela de Pressão Elétrica Philips Walita Daily 5L com Timer
Comentário: SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANELAS ELÉTRICAS.  COSTUMO USAR OUTRA PANELA PARA COZIMENTO DE ARROZ (JAPONESA), MAS LEVA MUITO TEMPO,  +/- 50 MINUTOS.  NESSA PANELA  É MUITO MAIS RÁPIDO, EXATAMENTE 6 MINUTOS.    EU RECOMENDO.
Avaliação: 4

Resultados para consulta por comentário:
Resultado 1:
Produto: Copo Acrílico Com Canudo 500ml Rocie
Comentário: produto mto bom, com essa garrafinh

In [8]:
from transformers import pipeline

# Carregar o pipeline de geração de texto
generator = pipeline('text-generation', model='gpt2')

# Função para gerar mensagem com base em comentário similar
# Alguém vai precisar aprimorar isso aqui, ele tá invocando entidades por enquanto (apagar esse comentário antes da entrega)
def gerar_resposta(comentario):
    # Gerar uma resposta com base no comentário encontrado
    resposta = generator(f"Baseado neste comentário: '{comentario}', gere um resumo.", max_length=100, num_return_sequences=1)
    return resposta[0]['generated_text']

# Exemplo de consulta
consulta = "Horrível."
resultados = buscar_no_faiss(consulta, tipo='comentario', k=1)  # Buscar 1 comentário mais próximo

# Gerar e exibir a resposta para o comentário mais próximo
if resultados:
    comentario_proximo = resultados[0]['comentário']
    mensagem_gerada = gerar_resposta(comentario_proximo)
    print(f"Comentário mais similar: {comentario_proximo}")
    print(f"Mensagem gerada: {mensagem_gerada}")
else:
    print("Nenhum comentário encontrado.")

h:\Github\Fatec\BACK-DOMROCK\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Comentário mais similar: produto mto bom, com essa garrafinha vc pode até servir água pro megazord. To pensando em vender minha tv pra comprar 1 garrafa dessa. RECOMENDO
Mensagem gerada: Baseado neste comentário: 'produto mto bom, com essa garrafinha vc pode até servir água pro megazord. To pensando em vender minha tv pra comprar 1 garrafa dessa. RECOMENDO', gere um resumo. RECOMENDO hacer tresa tresa: puede es, difídia hacer otra, su
